In this practice code, we are going to use Ozone dataset. Dictionary of this dataset can be found in https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/airquality.html

In [42]:
import pandas as pd
import numpy as np

In [44]:
url = "/Users/jcano/src/repos/SF-DAT-20/Data/ozone.csv"
OzoneData = pd.read_csv(url)

#### Explore the dataset and decide which variables suffer from missing data

In [53]:
OzoneData.describe()
#OzoneData.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
count,116.000000,146.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000
mean,42.129310,185.931507,9.957516,77.882353,6.993464,15.803922,0.202614,0.196078,0.202614,0.202614
std,32.987885,90.058422,3.523001,9.465270,1.416522,8.864520,0.403268,0.398332,0.403268,0.403268
min,1.000000,7.000000,1.700000,56.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,18.000000,115.750000,7.400000,72.000000,6.000000,8.000000,0.000000,0.000000,0.000000,0.000000
50%,31.500000,205.000000,9.700000,79.000000,7.000000,16.000000,0.000000,0.000000,0.000000,0.000000
75%,63.250000,258.750000,11.500000,85.000000,8.000000,23.000000,0.000000,0.000000,0.000000,0.000000
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000,1.000000,1.000000,1.000000,1.000000


Answer: Ozone and Solar.R are missing some data.

#### Let's drop rows that have missing values in all the columns you indentified above

Hint: in dropna() if you set how = 'all', it will only drop columns that are suffering from missing values at all varaibles you introduce in subset. If you want to get rid of the row that contains missing values in any of the variables you specify, then you shall set how = 'any'

df.dropna(how = 'all',subset = ['Var1','Var2','Var3'],inplace = True)

The above code will check if all 3 variables specified in df have missing values, if they all have missing values it will drop that row.

In [30]:
OzoneData.dropna(how="all", subset=["Ozone", "Solar.R"], inplace=True)

In [31]:
OzoneData.describe()

,Ozone,Solar.R,Wind,Temp,Month,Day
count,116.000000,146.000000,151.000000,151.000000,151.000000,151.000000
mean,42.129310,185.931507,9.941722,78.165563,7.019868,15.801325
std,32.987885,90.058422,3.524984,9.198138,1.406984,8.832531
min,1.000000,7.000000,1.700000,57.000000,5.000000,1.000000
25%,18.000000,115.750000,7.400000,73.000000,6.000000,8.000000
50%,31.500000,205.000000,9.700000,79.000000,7.000000,16.000000
75%,63.250000,258.750000,11.500000,85.000000,8.000000,23.000000
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000


#### Spoiler! If everything is going according to plan you shall be left by 151 observations. Also, it seemed like the first two variables had missing values. Now plese make a copy of your dataframe into a dataframe named OzoneImputeMean. Also, please use mean of the variables to fill in missing values in OzoneImputeMean

In [32]:

OzoneImputeMean = OzoneData
OzoneMissing = OzoneData.dropna(how="any", subset=["Ozone"], inplace=False)
OzoneImputeMean["Ozone"].fillna(value=OzoneMissing["Ozone"].mean(), inplace=True)
OzoneMissing = OzoneData.dropna(how="any", subset=["Solar.R"], inplace=False)
OzoneImputeMean["Solar.R"].fillna(value=OzoneMissing["Solar.R"].mean(), inplace=True)

In [33]:
#Check if missing values are filled
OzoneImputeMean.describe()


,Ozone,Solar.R,Wind,Temp,Month,Day
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,42.129310,185.931507,9.941722,78.165563,7.019868,15.801325
std,28.884028,88.544727,3.524984,9.198138,1.406984,8.832531
min,1.000000,7.000000,1.700000,57.000000,5.000000,1.000000
25%,21.000000,119.000000,7.400000,73.000000,6.000000,8.000000
50%,42.129310,197.000000,9.700000,79.000000,7.000000,16.000000
75%,46.500000,257.000000,11.500000,85.000000,8.000000,23.000000
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000


#### Now it's time for imputing using linear regression lines

In [46]:
#### Before we start let's define dummy variables for variable Month - don't worry about day!
MonthDummy = pd.get_dummies(OzoneData.Month, prefix = 'Month')
del MonthDummy['Month_9']
OzoneData = pd.concat([OzoneData, MonthDummy], axis=1)
OzoneData.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
0,41,190,7.4,67,5,1,1,0,0,0
1,36,118,8.0,72,5,2,1,0,0,0
2,12,149,12.6,74,5,3,1,0,0,0
3,18,313,11.5,62,5,4,1,0,0,0
4,NaN,NaN,14.3,56,5,5,1,0,0,0


In [36]:
#now let's explore correlation Matrix
OzoneData.corr()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
Ozone,1.000000,0.302970,-0.534163,0.630583,0.151089,-0.011472,-0.281111,-0.065820,0.251524,0.264054
Solar.R,0.302970,1.000000,-0.055581,0.271990,-0.073886,-0.147120,-0.023842,0.023896,0.175960,-0.073214
Wind,-0.534163,-0.055581,1.000000,-0.466032,-0.175317,0.042365,0.237781,0.046054,-0.144638,-0.166105
Temp,0.630583,0.271990,-0.466032,1.000000,0.397427,-0.136876,-0.637816,0.050753,0.318103,0.321680
Month,0.151089,-0.073886,-0.175317,0.397427,1.000000,-0.007727,-0.702257,-0.362131,-0.007201,0.355246
Day,-0.011472,-0.147120,0.042365,-0.136876,-0.007727,1.000000,0.011003,-0.017044,0.011471,0.011471
Month_5,-0.281111,-0.023842,0.237781,-0.637816,-0.702257,0.011003,1.000000,-0.242766,-0.247805,-0.247805
Month_6,-0.065820,0.023896,0.046054,0.050753,-0.362131,-0.017044,-0.242766,1.000000,-0.253080,-0.253080
Month_7,0.251524,0.175960,-0.144638,0.318103,-0.007201,0.011471,-0.247805,-0.253080,1.000000,-0.258333
Month_8,0.264054,-0.073214,-0.166105,0.321680,0.355246,0.011471,-0.247805,-0.253080,-0.258333,1.000000


#### What seems to be the list of best variables can define Ozone? how about Solar.R?

Answer: IT seems that Temp, Wind and Solar.R are the best variables to use.

In [50]:
# now let's use a regression model to predict Ozone. First drop NaN values in Ozone and save it in OzoneDroppedValues_Ozone
# the run a regression line on variables of interest and check significancy of your model
# if in a multi class dummy variable case, you see a few of the dummy variables are not significant but the rest are, 
#you shall either drop all or keep all. Otherwise, selecting the base dummy will become important
# use these variables ['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8'] to predict
from sklearn.linear_model import LinearRegression
from sklearn import feature_selection

OzoneDroppedValues_Ozone = OzoneData.dropna(how='any', subset=['Ozone'], inplace=False)
OzoneDroppedValues_Ozone.describe()
#OzoneData.describe()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
count,116.000000,111.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000
mean,42.129310,184.801802,9.862069,77.870690,7.198276,15.534483,0.224138,0.077586,0.224138,0.224138
std,32.987885,91.152302,3.574856,9.485486,1.475715,8.750087,0.418823,0.268680,0.418823,0.418823
min,1.000000,7.000000,2.300000,57.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,18.000000,113.500000,7.400000,71.000000,6.000000,8.000000,0.000000,0.000000,0.000000,0.000000
50%,31.500000,207.000000,9.700000,79.000000,7.000000,16.000000,0.000000,0.000000,0.000000,0.000000
75%,63.250000,255.500000,11.500000,85.000000,8.250000,22.000000,0.000000,0.000000,0.000000,0.000000
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000,1.000000,1.000000,1.000000,1.000000


In [66]:
# now fill in null values of OzoneData['Ozone'] by predicted values
lm = LinearRegression()
OzoneDroppedValues_Ozone = OzoneData.dropna(how='any', subset=['Ozone'], inplace=False)
X = OzoneDroppedValues_Ozone[['Wind','Temp','Month_5','Month_7','Month_8']]
y = OzoneDroppedValues_Ozone["Ozone"]
lm.fit(X,y)
pvals = feature_selection.f_regression(X,y)[1]
print pvals
#print lm.predict(X)
OzoneDroppedValues_Ozone["Ozone_predict"] = lm.predict(X)
OzoneDroppedValues_Ozone["Ozone"].fillna(value = OzoneDroppedValues_Ozone["Ozone_predict"], inplace=True)
OzoneDroppedValues_Ozone.describe()

[  9.27197390e-13   2.93189659e-18   9.47317968e-04   2.51873571e-03
   1.48141562e-03]


/Users/jcano/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8,Ozone_predict
count,116.000000,111.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000
mean,42.129310,184.801802,9.862069,77.870690,7.198276,15.534483,0.224138,0.077586,0.224138,0.224138,42.129310
std,32.987885,91.152302,3.574856,9.485486,1.475715,8.750087,0.418823,0.268680,0.418823,0.418823,25.584113
min,1.000000,7.000000,2.300000,57.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-16.116570
25%,18.000000,113.500000,7.400000,71.000000,6.000000,8.000000,0.000000,0.000000,0.000000,0.000000,22.384361
50%,31.500000,207.000000,9.700000,79.000000,7.000000,16.000000,0.000000,0.000000,0.000000,0.000000,42.772644
75%,63.250000,255.500000,11.500000,85.000000,8.250000,22.000000,0.000000,0.000000,0.000000,0.000000,61.034476
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000,1.000000,1.000000,1.000000,1.000000,98.470744


In [ ]:
# now fill in null values of OzoneData['Solar.R'] by predicted values


#### Now check your filled data - if your predicted values are more than maximum or less than minimum, replace them by max and min

## Bonus: In original question, replace missing values of Ozone and Solar.R by Single Imputation Regression Line with Error

In [ ]:
url = "https://raw.githubusercontent.com/ga-students/SF-DAT-20/master/Data/ozone.csv"
OzoneData = pd.read_csv(url)
OzoneData.dropna(how = 'all', subset = ['Ozone', 'Solar.R'], inplace = True)





